In [83]:
import catboost
import pandas as pd
import numpy as np
from typing import List
import pandas as pd
from scipy import stats
import datetime
import rpy2
from functools import reduce
from catboost import CatBoostRegressor
import time
import seaborn as sns
import ast
import numpy as np
from dateutil import parser
import xarray as xr
from kshape.core import kshape, zscore
from tslearn.clustering import KShape
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import catboost

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import time
import pickle
import warnings
warnings.filterwarnings("ignore")

In [49]:
df = pd.read_csv('Export\cdnow\cdnow_time_series_initial_2000.csv')
df.head()

,id,1997-01-01,1997-01-02,1997-01-03,1997-01-04,1997-01-05,1997-01-06,1997-01-07,1997-01-08,1997-01-09,...,1998-06-21,1998-06-22,1998-06-23,1998-06-24,1998-06-25,1998-06-26,1998-06-27,1998-06-28,1998-06-29,1998-06-30
0,6383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
df = pd.melt(df,id_vars = ['id'],var_name ='timestamp' ,value_name='value')
df.head()

,id,timestamp,value
0,6383,1997-01-01,NaN
1,6146,1997-01-01,NaN
2,5044,1997-01-01,NaN
3,10377,1997-01-01,NaN
4,6496,1997-01-01,NaN


In [32]:
df_freq = pd.read_csv('Export\cdnow\cdnow_frequency_2000.csv').loc[:,['Unnamed: 0','label']]
df_monet = pd.read_csv('Export\cdnow\cdnow_monetary_2000.csv').loc[:,['Unnamed: 0','label']]
df_recency = pd.read_csv('Export\cdnow\cdnow_recency_2000.csv').loc[:,['Unnamed: 0','label']]

In [36]:
df_monet.head()

,Unnamed: 0,label
0,6383,3
1,6146,3
2,5044,3
3,10377,3
4,6496,3


In [56]:
df_combined = df.merge(df_recency,left_on = 'id',right_on='Unnamed: 0') \
    .merge(df_freq,left_on = 'id',right_on='Unnamed: 0') \
    .merge(df_monet,left_on = 'id',right_on='Unnamed: 0') \
    .drop(columns =['Unnamed: 0_x','Unnamed: 0_y','Unnamed: 0'])
df_combined.columns = ['id','timestamp','value','recency','frequency','monetary']
df_combined.sample(5)

,id,timestamp,value,recency,frequency,monetary
561157,16205,1998-02-20,NaN,1,0,3
856949,19352,1997-10-03,NaN,2,0,3
588700,20719,1997-04-23,NaN,0,0,0
434456,7794,1998-01-22,NaN,0,1,3
258518,23497,1997-09-18,NaN,3,0,3


In [63]:
#For CDNow
df_combined['timestamp'] = df_combined['timestamp'].apply(lambda x:datetime.datetime.strptime(x, "%Y-%m-%d"))
df_combined.sample(2)

,id,timestamp,value,recency,frequency,monetary
264994,15118,1997-07-04,NaN,0,0,3
479349,952,1998-05-23,NaN,0,0,3


In [77]:
# number of months
test_period = 4
current_date = df_combined.timestamp.max()
df_test = df_combined[df_combined.timestamp >= current_date - pd.DateOffset(months = test_period )]
df_train = df_combined[df_combined.timestamp < current_date - pd.DateOffset(months = test_period )]

In [78]:
df_test.shape[0]/df_train.shape[0]

0.2907801418439716

In [79]:
y_train = df_train['value']
y_test = df_test['value']

X_train = df_train.drop('value', axis = 1)
X_test = df_test.drop('value', axis = 1)

## Shared prediction algorithm

In [ ]:
start_time = time.time()

cv = KFold(n_splits = 3, shuffle = True, random_state = 42)  

model = CatBoostRegressor(verbose = False, 
                           random_state=42)

param_grid = {
              'n_estimators': [200, 300],
              'max_depth': [3, 4], 
              'learning_rate': [0.03, 0.05, 0.07] 
             }

gs_cb = GridSearchCV(model, 
                  param_grid,
                  cv = cv,
                  scoring = 'neg_mean_squared_error')

gs_cb.fit(X_train, y_train)

train_RMSE = np.sqrt(-gs_cb.cv_results_['mean_train_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()
train_std = np.sqrt(gs_cb.cv_results_['std_train_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()

cv_RMSE = np.sqrt(-gs_cb.cv_results_['mean_test_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()
cv_std = np.sqrt(gs_cb.cv_results_['std_test_score'][gs_cb.cv_results_['rank_test_score'] == 1]).mean()

test_RMSE = np.sqrt(mean_squared_error(gs_cb.predict(X_test), y_test))

print('train RMSE: {} +/- {}'.format(np.round(train_RMSE, 2), np.round(train_std, 2)))
print('CV RMSE: {} +/- {}'.format(np.round(cv_RMSE, 2), np.round(cv_std, 2)))
print('test RMSE: {}'.format(np.round(test_RMSE, 2)))
print(gs_cb.best_params_)
print('time :{} min'.format(np.round((time.time() - start_time)/60, 2))) 

filename = 'CatBoost.sav'
pickle.dump(gs_cb, open(filename, 'wb'))